In [4]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
import geopandas as gpd
import xdem
import rasterio

DEM_folder = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/'

In [7]:
warp_raster_to_reference('/data/Aldhani/eoagritwin/et/Auxiliary/landcover/thuenen/CTM_GER_2021_rst_v201_COG.tif',
                         '/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/Acq_time/int_format/2019/Daily_AcqTime_maxLST_2019_April.tif',
                         f'{DEM_folder}reprojected/THUENEN_GER_LST_WARP.tif',
                         resampling='nearest')

Raster warped and saved to: /data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/THUENEN_GER_LST_WARP.tif


In [8]:
warp_raster_to_reference('/data/Aldhani/eoagritwin/et/Auxiliary/landcover/thuenen/CTM_GER_2021_rst_v201_COG.tif',
                         f'{DEM_folder}reprojected/FORCE_TILE_DUMMY.tif',
                         f'{DEM_folder}reprojected/THUENEN_GER_FORCE_WARP.tif',
                         resampling='nearest')

Raster warped and saved to: /data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/THUENEN_GER_FORCEWARP.tif


In [6]:
# make binary Thuenen mask
th_ds = gdal.Open(f'{DEM_folder}reprojected/THUENEN_GER_FORCE_WARP.tif')
th_arr = th_ds.GetRasterBand(1).ReadAsArray()
th_arr_ma = np.where(th_arr == -9999, 0 ,1)
npTOdisk(th_arr_ma, f'{DEM_folder}reprojected/THUENEN_GER_FORCE_WARP.tif', f'{DEM_folder}reprojected/THUENEN_GER_FORCE_WARP_BINARY.tif')

[-9999   200  1101  1102  1103  1201  1202  1300  1401  1402  1501  1502
  1602  1603  1611  1612  1613  1614  3001  3002  3003  3004  4001  4002
  4003]


In [7]:
# make binary Thuenen mask
th_ds = gdal.Open(f'{DEM_folder}reprojected/THUENEN_GER_LST_WARP.tif')
th_arr = th_ds.GetRasterBand(1).ReadAsArray()
th_arr_ma = np.where(th_arr == -9999, 0 ,1)
npTOdisk(th_arr_ma, f'{DEM_folder}reprojected/THUENEN_GER_LST_WARP.tif', f'{DEM_folder}reprojected/THUENEN_GER_LST_WARP_BINARY.tif')

In [ ]:
# now cut into FORCE TILES
force_grid = gpd.read_file('/data/Aldhani/eoagritwin/misc/FORCE_footprint/datacube-grid_DEU_10km.gpkg')

for product, name in zip([f'{DEM_folder}reprojected/THUENEN_GER_FORCE_WARP.tif'],
                          ['THUENEN_2021']):
    with rasterio.open(product) as src:
        varPath = f'{DEM_folder}Force_Tiles/{name}'
        if not os.path.exists(varPath):
            os.makedirs(varPath)
        for tileID in force_grid['Tile_ID']:
            tile = force_grid[force_grid['Tile_ID'] == tileID]
            # print(Vector.crs)
            out_image, out_transform=rasterio.mask.mask(src,tile.geometry,crop=True)
            out_meta=src.meta.copy() # copy the metadata of the source DEM
            
            out_meta.update({
                "driver":"Gtiff",
                "height":out_image.shape[1], # height starts with shape[1]
                "width":out_image.shape[2], # width starts with shape[2]
                "transform":out_transform
            })
                    
            with rasterio.open(f'{varPath}/{name}_{tileID}.tif','w',**out_meta) as dst:
                dst.write(out_image)